<a href="https://colab.research.google.com/github/aecins/tutorials/blob/main/se3_trajectory_interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install spatialmath-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 38.6 MB/s eta 0:00:00


In [48]:
import numpy as np
import spatialmath as sp

#Create two SE3 poses with different rotations and translations.
pose_1 = sp.SE3()

pose_2 = sp.SE3.Rt(
    t=[2, 2, 2],
    R=sp.SO3.AngleAxis(
        theta=75,
        v=[0, 1, 0],
        unit="deg",
    ),
)

print("Pose 1:")
print(pose_1)
print("Pose 2:")
print(pose_2)

print("Distance between origins of the poses:")
print(np.linalg.norm(pose_1.t - pose_2.t))

Pose 1:
   1         0         0         0         
   0         1         0         0         
   0         0         1         0         
   0         0         0         1         

Pose 2:
   0.2588    0         0.9659    2         
   0         1         0         2         
  -0.9659    0         0.2588    2         
   0         0         0         1         

Distance between origins of the poses:
3.4641016151377544


In [49]:
pose_1_inv = pose_1.inv()
pose_2_inv = pose_2.inv()

print("Pose 1 inverse:")
print(pose_1_inv)
print("Pose 2 inverse:")
print(pose_2_inv)

print("Distance between origins of the inverses of poses:")
print(np.linalg.norm(pose_1.inv().t - pose_2.inv().t))

Pose 1 inverse:
   1         0         0         0         
   0         1         0         0         
   0         0         1         0         
   0         0         0         1         

Pose 2 inverse:
   0.2588    0        -0.9659    1.414     
   0         1         0        -2         
   0.9659    0         0.2588   -2.449     
   0         0         0         1         

Distance between origins of the inverses of poses:
3.4641016151377544


In [50]:
# Interpolate between the original poses.
pose_1_2_interp = pose_1.interp(pose_2, 0.5)

print("Interpolation of the original poses:")
print(pose_1_2_interp)

# Take inverse of the interpolation between inverted poses.
pose_1_2_inv_interp = pose_1.inv().interp(pose_2.inv(), 0.5)
pose_1_2_inv_interp_inv = pose_1_2_inv_interp.inv()
print("Inverse of the interpolation of the inverted poses:")
print(pose_1_2_inv_interp_inv)

print("Distance between origins of the interpolated poses:")
print(f"  {np.linalg.norm(pose_1_2_interp.t - pose_1_2_inv_interp_inv.t)}")
print("Angular distance between rotations of the interpolated poses:")
print(f"  {pose_1_2_interp.UnitQuaternion().angdist(pose_1_2_inv_interp_inv.UnitQuaternion())}")

Interpolation of the original poses:
   0.7934    0         0.6088    1         
   0         1         0         1         
  -0.6088    0         0.7934    1         
   0         0         0         1         

Inverse of the interpolation of the inverted poses:
   0.7934    0         0.6088    0.1846    
   0         1         0         1         
  -0.6088    0         0.7934    1.402     
   0         0         0         1         

Distance between origins of the interpolated poses:
  0.9091681026273741
Angular distance between rotations of the interpolated poses:
  0.0


This demonstates that in a general case the interpolation of two $SE3$ poses is not equal to the inverse of the interpolation of the inverted poses i.e.:
$$
interp(P_1, P_2, \alpha) \neq interp(P_1^{-1},P_2^{-1}, \alpha)^{-1}
$$

where $interp(...)$ interpolates between two $SE3$ poses such that SLERP is used for rotation and linear interpolation for translation.


In [51]:
#Create two SE3 poses with different rotations but identical translations
pose_1 = sp.SE3.Rt(
    t=[1, 1, 1],
    R=sp.SO3.AngleAxis(
        theta=45,
        v=[1, 1, 1],
        unit="deg",
    ),
)

pose_2 = sp.SE3.Rt(
    t=[1, 1, 1],
    R=sp.SO3.AngleAxis(
        theta=45,
        v=[1, 1, 1],
        unit="deg",
    ),
)

In [52]:
# Interpolate between the original poses.
pose_1_2_interp = pose_1.interp(pose_2, 0.5)

print("Interpolation of the original poses:")
print(pose_1_2_interp)

# Take inverse of the interpolation between inverted poses.
pose_1_2_inv_interp = pose_1.inv().interp(pose_2.inv(), 0.5)
pose_1_2_inv_interp_inv = pose_1_2_inv_interp.inv()
print("Inverse of the interpolation of the inverted poses:")
print(pose_1_2_inv_interp_inv)

print("Distance between origins of the interpolated poses:")
print(f"  {np.linalg.norm(pose_1_2_interp.t - pose_1_2_inv_interp_inv.t)}")
print("Angular distance between rotations of the interpolated poses:")
print(f"  {pose_1_2_interp.UnitQuaternion().angdist(pose_1_2_inv_interp_inv.UnitQuaternion())}")

Interpolation of the original poses:
   0.8047   -0.3106    0.5059    1         
   0.5059    0.8047   -0.3106    1         
  -0.3106    0.5059    0.8047    1         
   0         0         0         1         

Inverse of the interpolation of the inverted poses:
   0.8047   -0.3106    0.5059    1         
   0.5059    0.8047   -0.3106    1         
  -0.3106    0.5059    0.8047    1         
   0         0         0         1         

Distance between origins of the interpolated poses:
  0.0
Angular distance between rotations of the interpolated poses:
  0.0


This demonstates that in a special case where two poses have same rotations, the interpolation of two poses is equal to the inverse of the interpolation of the inverted poses i.e.:
$$
interp(P_1, P_2, \alpha) == interp(P_1^{-1},P_2^{-1}, \alpha)^{-1} \iff P_1.rotation() == P_2.rotation()
$$

This suggests that for a 6DoF trajectory of a frame that is sampled densely enough (relative rotation between consecutive trajectory samples is small) - interpolating the original trajectory will give results approximately equal to taking the inverse of the interpolation of the inverse trajectory.